In [ ]:
import sys
import os

from torch.utils.data import DataLoader

import matplotlib.pyplot as plt
import torchvision.transforms as transforms

current_dir = os.getcwd()
src_dir = os.path.join(current_dir, '..')

if src_dir not in sys.path:
    sys.path.append(src_dir)

from src.data.classification import TumorClassificationDataset
from src.enums import DataSplit

In [ ]:
import os
from torchvision import transforms
from torch.utils.data import DataLoader

DATASETS_ROOT = os.path.join(current_dir, '..', 'datasets')
BATCH_SIZE = 4

# Define the transformation pipeline for the dataset
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize all images to 224x224
    transforms.ToTensor(),  # Convert images to tensor
    # Add any other transforms here, e.g., normalization
])

# Create an instance of the TumorBinaryClassificationDataset
dataset = TumorClassificationDataset(root_dir=DATASETS_ROOT, split=DataSplit.TRAIN, transform=transform)

# Create a data loader to load the dataset in batches
data_loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)

In [ ]:
# Step 1: Prepare the Data
train_dataset = TumorClassificationDataset(root_dir=DATASETS_ROOT, split=DataSplit.TRAIN, transform=transform)
test_dataset = TumorClassificationDataset(root_dir=DATASETS_ROOT, split=DataSplit.TEST, transform=transform)


# Define data loaders for training and testing sets
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
from torch import nn
class ResidualBlockMulti(nn.Module):
    """
    A residual block module for ResNet.
       
    Args:
        in_channels (int): Number of input channels.
        out_channels (int): Number of output channels.
        stride (int, optional): Stride value for the convolutional layers. Default is 1.
        downsample (nn.Module, optional): Downsample module to be applied to the input. Default is None.

    Attributes:
        conv1 (nn.Sequential): First convolutional layer sequence.
        conv2 (nn.Sequential): Second convolutional layer sequence.
        downsample (nn.Module): Downsample module to be applied to the input.
        relu (nn.ReLU): ReLU activation function.
        out_channels (int): Number of output channels.

    """

    expansion = 1

    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlockMulti, self).__init__()
        
        # Define the first convolutional layer sequence
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU()
        )

        # Define the second convolutional layer sequence
        self.conv2 = nn.Sequential(
            nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(out_channels)
        )

        # Set the downsample module
        self.downsample = downsample

        # Define the ReLU activation function
        self.relu = nn.ReLU()

        # Set the number of output channels
        self.out_channels = out_channels

    def forward(self, x):
        """
        Forward pass of the residual block.

        Args:
            x (torch.Tensor): Input tensor.

        Returns:
            torch.Tensor: Output tensor.

        """
        # Store the input tensor as the residual
        residual = x

        # Apply the first convolutional layer sequence
        out = self.conv1(x)

        # Apply the second convolutional layer sequence
        out = self.conv2(out)

        # Apply the downsample module if it exists
        if self.downsample:
            residual = self.downsample(x)

        # Add the residual to the output and apply the ReLU activation function
        out += residual
        out = self.relu(out)

        return out

In [ ]:
class ResNetMulti(nn.Module):
    """
    ResNet class for image classification.

    Args:
        block (nn.Module): The residual block module.
        layers (list): List of integers representing the number of residual blocks in each layer.
        num_classes (int, optional): Number of output classes. Defaults to 10.
    """

    def __init__(self, block, layers, num_classes=10):
        super(ResNetMulti, self).__init__()
        self.inplanes = 64
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3),  # 7x7 convolutional layer with stride 2 and padding 3
            nn.BatchNorm2d(64),  # Batch normalization layer for 64 channels
            nn.ReLU()  # ReLU activation function
        )
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)  # Max pooling layer with kernel size 3, stride 2, and padding 1
        self.layer0 = self._make_layer(block, 64, layers[0], stride=1)  # First residual layer
        self.layer1 = self._make_layer(block, 128, layers[1], stride=2)  # Second residual layer
        self.layer2 = self._make_layer(block, 256, layers[2], stride=2)  # Third residual layer
        self.layer3 = self._make_layer(block, 512, layers[3], stride=2)  # Fourth residual layer
        self.avgpool = nn.AvgPool2d(7, stride=1)  # Average pooling layer with kernel size 7 and stride 1
        self.fc = nn.Linear(512, num_classes)  # Fully connected layer with 512 input features and num_classes output features

    def _make_layer(self, block, planes, blocks, stride=1):
        """
        Helper method to create a residual layer.

        Args:
            block (nn.Module): The residual block module.
            planes (int): Number of output channels.
            blocks (int): Number of residual blocks in the layer.
            stride (int, optional): Stride value. Defaults to 1.

        Returns:
            nn.Sequential: Sequential module containing the residual blocks.
        """
        downsample = None
        if stride != 1 or self.inplanes != planes:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes, kernel_size=1, stride=stride),  # 1x1 convolutional layer with stride
                nn.BatchNorm2d(planes)  # Batch normalization layer for planes channels
            )
        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        """
        Forward pass of the ResNet model.

        Args:
            x (torch.Tensor): Input tensor.

        Returns:
            torch.Tensor: Output tensor.
        """
        x = self.conv1(x)  # Apply the first convolutional layer sequence
        x = self.maxpool(x)  # Apply max pooling
        x = self.layer0(x)  # Apply the first residual layer
        x = self.layer1(x)  # Apply the second residual layer
        x = self.layer2(x)  # Apply the third residual layer
        x = self.layer3(x)  # Apply the fourth residual layer

        x = self.avgpool(x)  # Apply average pooling
        x = x.view(x.size(0), -1)  # Flatten the tensor
        x = self.fc(x)  # Apply the fully connected layer

        return x

In [ ]:
# # Modified Version of the ResNetMulti Class
# class ResNetMulti(nn.Module):
#     """
#     ResNet class for image classification.

#     Args:
#         block (nn.Module): The residual block module.
#         layers (list): List of integers representing the number of residual blocks in each layer.
#         num_classes (int, optional): Number of output classes. Defaults to 10.
#     """

#     def __init__(self, block, layers, num_classes=10):
#         super(ResNetMulti, self).__init__()
#         self.inplanes = 64
#         self.conv1 = nn.Sequential(
#             nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3),  # 7x7 convolutional layer with stride 2 and padding 3
#             nn.BatchNorm2d(64),  # Batch normalization layer for 64 channels
#             nn.ReLU()  # ReLU activation function
#         )
#         self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)  # Max pooling layer with kernel size 3, stride 2, and padding 1
#         self.layer0 = self._make_layer(block, 64, layers[0], stride=1)  # First residual layer
#         self.layer1 = self._make_layer(block, 128, layers[1], stride=2)  # Second residual layer
#         self.layer2 = self._make_layer(block, 256, layers[2], stride=2)  # Third residual layer
#         self.layer3 = self._make_layer(block, 512, layers[3], stride=2)  # Fourth residual layer
#         self.avgpool = nn.AvgPool2d(7, stride=1)  # Average pooling layer with kernel size 7 and stride 1
#         self.fc = nn.Linear(512, num_classes)  # Fully connected layer with 512 input features and num_classes output features

#     def _make_layer(self, block, planes, blocks, stride=1):
#         """
#         Helper method to create a residual layer.

#         Args:
#             block (nn.Module): The residual block module.
#             planes (int): Number of output channels.
#             blocks (int): Number of residual blocks in the layer.
#             stride (int, optional): Stride value. Defaults to 1.

#         Returns:
#             nn.Sequential: Sequential module containing the residual blocks.
#         """
#         downsample = None
#         if stride != 1 or self.inplanes != planes * block.expansion:
#             downsample = nn.Sequential(
#                 nn.Conv2d(self.inplanes, planes * block.expansion, kernel_size=1, stride=stride, bias=False),
#                 nn.BatchNorm2d(planes * block.expansion),
#             )

#         layers = []
#         layers.append(block(self.inplanes, planes, stride, downsample))
#         self.inplanes = planes
#         for i in range(1, blocks):
#             layers.append(block(self.inplanes, planes))

#         return nn.Sequential(*layers)

#     def forward(self, x):
#         """
#         Forward pass of the ResNet model.

#         Args:
#             x (torch.Tensor): Input tensor.

#         Returns:
#             torch.Tensor: Output tensor.
#         """
#         x = self.conv1(x)  # Apply the first convolutional layer sequence
#         x = self.maxpool(x)  # Apply max pooling
#         x = self.layer0(x)  # Apply the first residual layer
#         x = self.layer1(x)  # Apply the second residual layer
#         x = self.layer2(x)  # Apply the third residual layer
#         x = self.layer3(x)  # Apply the fourth residual layer

#         x = self.avgpool(x)  # Apply average pooling
#         x = x.view(x.size(0), -1)  # Flatten the tensor
#         x = self.fc(x)  # Apply the fully connected layer

#         return x

In [ ]:
import torch.nn as nn
import torch
import torch.nn as nn
import torch.optim as optim
    
# Step 2: Define the Model

class TumorMultiClassifier(nn.Module):
    def __init__(self, num_classes):
        super(TumorMultiClassifier, self).__init__()
        # Load ResNet model
        self.model = ResNetMulti(ResidualBlockMulti, [2, 2, 2, 2], num_classes=num_classes)
        self.model.fc = nn.Linear(512, num_classes)  # Replace the last fully connected layer for multi-class classification

    def forward(self, x):
        return self.model(x)

In [ ]:

# # Step 3: Define the Loss Function and Optimizer
# num_classes = 4  # Number of classes in the tumor dataset
# model = TumorMultiClassifier(num_classes=num_classes)
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=0.001)

# # Step 4: Training Loop
# # Set the number of training epochs
# num_epochs = 10

# # Determine the device to use for computation (GPU if available, else CPU)
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# # Move the model to the chosen device
# model.to(device)

# # Start the training loop
# for epoch in range(num_epochs):
#     # Set the model to training mode
#     model.train()
    
#     # Initialize the running loss for this epoch
#     running_loss = 0.0
    
#     # Iterate over the batches of images and labels in the training data loader
#     for images, labels in train_loader:
#         # Move the images and labels to the chosen device
#         images, labels = images.to(device), labels.to(device)
        
#         # Zero the gradients of the model parameters
#         optimizer.zero_grad()
        
#         # Forward pass: compute the model outputs for the current batch of images
#         outputs = model(images)
        
#         # Compute the loss between the model outputs and the actual labels
#         loss = criterion(outputs, labels)
        
#         # Backward pass: compute the gradients of the loss with respect to the model parameters
#         loss.backward()
        
#         # Update the model parameters
#         optimizer.step()
        
#         # Update the running loss
#         running_loss += loss.item() * images.size(0)
    
#     # Compute the average loss for this epoch
#     epoch_loss = running_loss / len(train_dataset)
    
#     # Print the epoch number and average loss for this epoch
#     print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')

# # Step 5: Evaluation
# # Set the model to evaluation mode. This affects layers like dropout and batchnorm.
# model.eval()

# # Initialize counters for the total number of images and the number of correctly classified images
# correct = 0
# total = 0

# # Disable gradient computation since we don't need it for evaluation
# with torch.no_grad():
#     # Iterate over the batches of images and labels in the test data loader
#     for images, labels in test_loader:
#         # Move the images and labels to the chosen device
#         images, labels = images.to(device), labels.to(device)
        
#         # Forward pass: compute the model outputs for the current batch of images
#         outputs = model(images)
        
#         # Get the predicted classes for the current batch of images
#         _, predicted = torch.max(outputs.data, 1)
        
#         # Update the total number of images
#         total += labels.size(0)
        
#         # Update the number of correctly classified images
#         correct += (predicted == labels).sum().item()

# # Compute the accuracy as the percentage of correctly classified images
# accuracy = 100 * correct / total

# # Print the accuracy on the test set
# print(f'Accuracy on the test set: {accuracy:.2f}%')

In [ ]:
# import torch.nn as nn
# import torch
# import torch.nn as nn
# import torch.optim as optim
 
# # Modified Version of the ResNetMulti Class
   
# # Step 2: Define the Model

# class TumorMultiClassifier(nn.Module):
#     def __init__(self, num_classes):
#         super(TumorMultiClassifier, self).__init__()
#         # Load ResNet model
#         self.model = ResNetMulti(ResidualBlockMulti, [2, 2, 2, 2], num_classes=num_classes)
#         self.model.fc = nn.Linear(512, num_classes)  # Replace the last fully connected layer for multi-class classification

#     def forward(self, x):
#         return self.model(x)

In [ ]:
# # Modified Version of the ResNetMulti Class


# # Step 3: Define the Loss Function and Optimizer
# num_classes = 4  # Number of classes in the tumor dataset
# model = TumorMultiClassifier(num_classes=num_classes)
# criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)


# # Step 4: Training Loop
# # Set the number of training epochs
# num_epochs = 10

# # Determine the device to use for computation (GPU if available, else CPU)
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# # Move the model to the chosen device
# model.to(device)

# # Start the training loop
# for epoch in range(num_epochs):
#     # Set the model to training mode
#     model.train()
    
#     # Initialize the running loss for this epoch
#     running_loss = 0.0
    
#     # Iterate over the batches of images and labels in the training data loader
#     for images, labels in train_loader:
#         # Move the images and labels to the chosen device
#         images, labels = images.to(device), labels.to(device)
        
#         # Zero the gradients of the model parameters
#         optimizer.zero_grad()
        
#         # Forward pass: compute the model outputs for the current batch of images
#         outputs = model(images)
        
#         # Compute the loss between the model outputs and the actual labels
#         loss = criterion(outputs, labels)
        
#         # Backward pass: compute the gradients of the loss with respect to the model parameters
#         loss.backward()
        
#         # Update the model parameters
#         optimizer.step()
        
#         # Update the running loss
#         running_loss += loss.item() * images.size(0)
    
#     # Adjust the learning rate
#     scheduler.step()
    
#     # Compute the average loss for this epoch
#     epoch_loss = running_loss / len(train_dataset)
    
#     # Print the epoch number and average loss for this epoch
#     print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')

# # Step 5: Evaluation
# # Set the model to evaluation mode. This affects layers like dropout and batchnorm.
# model.eval()

# # Initialize counters for the total number of images and the number of correctly classified images
# correct = 0
# total = 0

# # Disable gradient computation since we don't need it for evaluation
# with torch.no_grad():
#     # Iterate over the batches of images and labels in the test data loader
#     for images, labels in test_loader:
#         # Move the images and labels to the chosen device
#         images, labels = images.to(device), labels.to(device)
        
#         # Forward pass: compute the model outputs for the current batch of images
#         outputs = model(images)
        
#         # Get the predicted classes for the current batch of images
#         _, predicted = torch.max(outputs.data, 1)
        
#         # Update the total number of images
#         total += labels.size(0)
        
#         # Update the number of correctly classified images
#         correct += (predicted == labels).sum().item()

# # Compute the accuracy as the percentage of correctly classified images
# accuracy = 100 * correct / total

# # Print the accuracy on the test set
# print(f'Accuracy on the test set: {accuracy:.2f}%')

In [ ]:
# def save_model(model, save_path):
#     torch.save(model.state_dict(), save_path)

# save_path = '../model_registry/tumor_resnetmulticlassifier.pth'
# save_model(model, save_path)

In [ ]:
#load model
num_classes = 4
model_load = TumorMultiClassifier(num_classes=num_classes)
model_load.load_state_dict(torch.load('../model_registry/tumor_resnetmulticlassifier.pth'))

In [ ]:
# define the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 

# Step 5: Evaluation
# Set the model to evaluation mode. This affects layers like dropout and batchnorm.
model_load.eval()

# Initialize counters for the total number of images and the number of correctly classified images
correct = 0
total = 0

# Disable gradient computation since we don't need it for evaluation
with torch.no_grad():
    # Iterate over the batches of images and labels in the test data loader
    for images, labels in test_loader:
        # Move the images and labels to the chosen device
        images, labels = images.to(device), labels.to(device)
        
        # Forward pass: compute the model outputs for the current batch of images
        outputs = model_load(images)
        
        # Get the predicted classes for the current batch of images
        _, predicted = torch.max(outputs.data, 1)
        
        # Update the total number of images
        total += labels.size(0)
        
        # Update the number of correctly classified images
        correct += (predicted == labels).sum().item()

# Compute the accuracy as the percentage of correctly classified images
accuracy = 100 * correct / total

# Print the accuracy on the test set
print(f'Accuracy on the test set: {accuracy:.2f}%')

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

# get classification report
y_true = []
y_pred = []

# Disable gradient computation since we don't need it for evaluation

with torch.no_grad():
    # Iterate over the batches of images and labels in the test data loader
    for images, labels in test_loader:
        # Move the images and labels to the chosen device
        images, labels = images.to(device), labels.to(device)
        
        # Forward pass: compute the model outputs for the current batch of images
        outputs = model_load(images)
        
        # Get the predicted classes for the current batch of images
        _, predicted = torch.max(outputs.data, 1)
        
        y_true += labels.cpu().numpy().tolist()
        y_pred += predicted.cpu().numpy().tolist()

In [ ]:
print(classification_report(y_true, y_pred))

# get confusion matrix
print(confusion_matrix(y_true, y_pred))

In [ ]:
# print confusion matrix along with labels
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

# Define the class labels
class_labels = ['Glioma', 'Meningioma', 'No Tumor', 'Pituitary']

# Create a confusion matrix
cm = confusion_matrix(y_true, y_pred)

# Convert the confusion matrix to a DataFrame
cm_df = pd.DataFrame(cm, index=class_labels, columns=class_labels)

# Create a heatmap from the DataFrame
plt.figure(figsize=(8, 6))
sns.heatmap(cm_df, annot=True, cmap='Blues', fmt='g')
plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()